# Timesheet Analysis

The goal of this notebook is to analyze my timesheet of working hours

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

In [5]:
# Connecting to Google Spreadsheet API and getting the desired spreadsheet

scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name('/home/aiquis/EI/timesheet/client_secret.json', scope)
client = gspread.authorize(creds)

sheet = client.open("Controle de ponto - EI").sheet1

           Data Hora Entrada Hora Saída                     Obs
0    01/06/2016     08:15:00      19:37                        
1    02/06/2016     08:26:00   17:31:00                        
2    03/06/2016     08:08:00   21:31:00                        
3    04/06/2016                                                
4    05/06/2016                                                
5    06/06/2016     09:31:00   17:50:00                        
6    07/06/2016     07:59:00   19:00:00                        
7    08/06/2016     09:57:00   17:21:00                        
8    09/06/2016     09:32:00   17:42:00                        
9    10/06/2016     08:42:00   19:42:00  Trabalhei de casa (BH)
10   11/06/2016                                                
11   12/06/2016                                                
12   13/06/2016     08:46:00   17:02:00                        
13   14/06/2016     10:08:00   19:10:00                        
14   15/06/2016     09:47:00   19:09:00 

In [17]:
# Storing the content of the spreadsheet on a DataFrame

df = pd.DataFrame(sheet.get_all_records(), columns = ['Data', 'Hora Entrada', 'Hora Saída', 'Obs'])

df.columns = ['data', 'hora_entrada', 'hora_saida', 'obs']

(419, 4)
